In [2]:
import torch
import torchvision

In [3]:
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465),
                         (0.2023, 0.1994, 0.2010))
])
train_dataset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform
)
test_dataset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform
)

#from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=128, shuffle=False)
##TODO: Use dataloader on train and test dataset for future training
## each time only one batch will be fed into the network

Files already downloaded and verified
Files already downloaded and verified


In [24]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
## TODO: Please construct a network using 2 conv, 2 ReLU, 2 Maxpool Layer to half the size, 2 Fully Connected layers
## and define the forward function
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1=nn.Conv2d(3,32,kernel_size=3, padding=1)
        self.conv2=nn.Conv2d(32,64,kernel_size=3, padding=1)
        self.pool=nn.MaxPool2d(2,2)
        self.fc1=nn.Linear(64*8*8,100);
        self.fc2=nn.Linear(100,10);
    def forward(self, x):
        x=self.pool(F.relu(self.conv1(x)))
        x=self.pool(F.relu(self.conv2(x)))
        x=x.view(x.shape[0],-1)
        x=self.fc2(F.relu(self.fc1(x)))
        return x
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleCNN().to(device)
## TODO: Please manually calculate the number of trainable parameters you construct and compare with by writing a code

## TODO: Define your own loss function and optimizer
crit=nn.CrossEntropyLoss();
#optimizer=optim.Adam(model.parameters(),lr=0.001)


In [25]:
data,label=next(iter(train_dataloader))

In [26]:
device

device(type='cuda')

In [27]:
model(data.to(device)).shape

torch.Size([128, 10])

In [13]:
for epoch in range(5):  # train for 5 epochs
    model.train()
    running_loss = 0.0
    running_accuracy = 0.0
    size=0
    count=0
    for images, labels in train_dataloader:
        images=images.to(device)
        labels=labels.to(device)
        pred=model(images)        
        loss=crit(pred,labels)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        size+=images.shape[0]
        count+=1
        running_loss+=loss.item()
        running_accuracy+=(pred.argmax(dim=1)==labels).float().sum().item()
        ##TODO: Training the model and calculate the loss and accuracy
    print(f"Epoch {epoch+1}, training Loss: {running_loss/count:.4f}")
    print(f"Epoch {epoch+1}, training Acc: {running_accuracy/size:.4f}")
    model.eval()
    with torch.no_grad():
        running_loss = 0.0
        running_accuracy = 0.0
        size=0
        count=0
        for images, labels in test_dataloader:
            images=images.to(device)
            labels=labels.to(device)
            images=data.to(device)
            labels=label.to(device)
            pred=model(images)
            loss=crit(pred,labels)
            size+=images.shape[0]
            count+=1
            running_loss+=loss.item()
            running_accuracy+=(pred.argmax(dim=1)==labels).float().sum().item()
    print(f"Epoch {epoch+1}, testing Loss: {running_loss/count:.4f}")
    print(f"Epoch {epoch+1}, tesing Acc: {running_accuracy/size:.4f}")
        ## TODO: Test the model on test data by starting with model.eval()


Epoch 1, training Loss: 1.3534
Epoch 1, training Acc: 0.5187
Epoch 1, testing Loss: 1.0771
Epoch 1, tesing Acc: 0.5938
Epoch 2, training Loss: 0.9804
Epoch 2, training Acc: 0.6576
Epoch 2, testing Loss: 0.9005
Epoch 2, tesing Acc: 0.7266
Epoch 3, training Loss: 0.8413
Epoch 3, training Acc: 0.7062
Epoch 3, testing Loss: 0.7347
Epoch 3, tesing Acc: 0.7578
Epoch 4, training Loss: 0.7426
Epoch 4, training Acc: 0.7396
Epoch 4, testing Loss: 0.5967
Epoch 4, tesing Acc: 0.8047
Epoch 5, training Loss: 0.6590
Epoch 5, training Acc: 0.7723
Epoch 5, testing Loss: 0.5859
Epoch 5, tesing Acc: 0.8047


In [14]:
torch.save(model.state_dict(), "CIFAR10_model.pth")

In [28]:
def zeropower_via_newtonschulz5(G, steps=3, eps=1e-7):
    """
    Newton-Schulz iteration to compute the zeroth power / orthogonalization of G. We opt to use a
    quintic iteration whose coefficients are selected to maximize the slope at zero. For the purpose
    of minimizing steps, it turns out to be empirically effective to keep increasing the slope at
    zero even beyond the point where the iteration no longer converges all the way to one everywhere
    on the interval. This iteration therefore does not produce UV^T but rather something like US'V^T
    where S' is diagonal with S_{ii}' \sim Uniform(0.5, 1.5), which turns out not to hurt model
    performance at all relative to UV^T, where USV^T = G is the SVD.
    """
    assert len(G.shape) == 2
    a, b, c = (3.4445, -4.7750,  2.0315)
    X = G.bfloat16()
    X /= (X.norm() + eps) # ensure top singular value <= 1
    if G.size(0) > G.size(1):
        X = X.T
    for _ in range(steps):
        A = X @ X.T
        B = b * A + c * A @ A
        X = a * X + B @ X
    if G.size(0) > G.size(1):
        X = X.T
    return X

class Muon(torch.optim.Optimizer):
    def __init__(self, params, lr=1e-3, momentum=0, nesterov=False):
        if lr < 0.0:
            raise ValueError(f"Invalid learning rate: {lr}")
        if momentum < 0.0:
            raise ValueError(f"Invalid momentum value: {momentum}")
        if nesterov and momentum <= 0:
            raise ValueError("Nesterov momentum requires a momentum")
        defaults = dict(lr=lr, momentum=momentum, nesterov=nesterov)
        super().__init__(params, defaults)

    def step(self):
        for group in self.param_groups:
            lr = group["lr"]
            momentum = group["momentum"]
            for p in group["params"]:
                g = p.grad
                if g is None:
                    continue
                state = self.state[p]

                if "momentum_buffer" not in state.keys():
                    state["momentum_buffer"] = torch.zeros_like(g)
                buf = state["momentum_buffer"]
                buf.mul_(momentum).add_(g)
                g = g.add(buf, alpha=momentum) if group["nesterov"] else buf

                p.data.mul_(len(p.data)**0.5 / p.data.norm()) # normalize the weight
                update = zeropower_via_newtonschulz5(g.reshape(len(g), -1)).view(g.shape) # whiten the update
                p.data.add_(update, alpha=-lr) # take a step

In [29]:
# collect parameters
muon_params = list(model.fc1.parameters()) + list(model.fc2.parameters())
adam_params = [p for n, p in model.named_parameters() if not any(layer in n for layer in ["fc1", "fc2"])]
# example Adam optimizer
adam = torch.optim.Adam(adam_params, lr=1e-3)

# assume you have a Muon optimizer class (like from Muon repo)
#from muon import Muon  # <- replace with actual import
muon = Muon(muon_params, lr=0.02, momentum=0.95, nesterov=True)


In [ ]:
for epoch in range(15):
    model.train()
    running_loss = 0.0
    running_accuracy = 0.0
    size=0
    count=0
    for images, labels in train_dataloader:
        images, labels = images.to(device), labels.to(device)

        # zero grads
        adam.zero_grad()
        muon.zero_grad()

        # forward
        pred = model(images)
        loss = crit(pred, labels)

        # backward
        loss.backward()

        # step both optimizers
        adam.step()
        muon.step()
        size+=images.shape[0]
        count+=1
        running_loss+=loss.item()
        running_accuracy+=(pred.argmax(dim=1)==labels).float().sum().item()
            ##TODO: Training the model and calculate the loss and accuracy
    print(f"Epoch {epoch+1}, training Loss: {running_loss/count:.4f}")
    print(f"Epoch {epoch+1}, training Acc: {running_accuracy/size:.4f}")

Epoch 1, training Loss: 0.7807
Epoch 1, training Acc: 0.7315
